# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv = '../output_data/cities.csv'
weather_data = pd.read_csv(csv)
weather_data.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Bluff,0,NZ,1592509554,83,-46.60,168.33,3.89,1.34
1,1,Karpogory,81,RU,1592509554,63,64.00,44.45,12.80,1.98
2,2,Albany,39,US,1592509503,58,42.60,-73.97,31.11,0.45
3,3,Port Elizabeth,0,ZA,1592509554,82,-33.92,25.57,14.00,3.10
4,4,Ushuaia,24,AR,1592509554,64,-54.80,-68.30,2.00,3.33


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
weather_data


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Bluff,0,NZ,1592509554,83,-46.60,168.33,3.89,1.34
1,1,Karpogory,81,RU,1592509554,63,64.00,44.45,12.80,1.98
2,2,Albany,39,US,1592509503,58,42.60,-73.97,31.11,0.45
3,3,Port Elizabeth,0,ZA,1592509554,82,-33.92,25.57,14.00,3.10
4,4,Ushuaia,24,AR,1592509554,64,-54.80,-68.30,2.00,3.33
...,...,...,...,...,...,...,...,...,...,...
533,533,Hervey Bay,16,AU,1592509637,95,-25.30,152.85,16.67,6.26
534,534,Srostki,28,RU,1592509637,75,52.42,85.70,11.84,3.10
535,535,Aden,53,YE,1592509523,64,12.78,45.04,31.38,2.67
536,536,Iquitos,75,PE,1592509637,70,-3.75,-73.25,30.00,2.60


In [4]:
# Store 'Lat' and 'Lng' into  locations 
weather_data = weather_data.dropna()
intensity = weather_data['Humidity'].max()
cent= (0, 0)
weather_data = weather_data.loc[weather_data["Max Temp"] >= 0,:]
locations = weather_data[["Lat", "Lng"]].astype(float)
heat_data = weather_data["Max Temp"].astype(float)
fig = gmaps.figure(center=cent, zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=heat_data, 
                                 dissipating=False, max_intensity=intensity, point_radius = 5)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
# Narrow down the cities to fit weather conditions
vacation_data = weather_data.loc[(weather_data["Max Temp"] >= 30) & 
                                 (weather_data["Max Temp"] < 40) &
                                 (weather_data["Humidity"] < 50) & 
                                 (weather_data["Cloudiness"] < 50) & 
                                 (weather_data["Wind Speed"] < 3)]
vacation_data = vacation_data.reset_index(drop=True)
vacation_data

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,89,Sinaloa,0,MX,1592509359,34,25.00,-107.50,36.67,2.24
1,234,Kharan,0,PK,1592509591,10,28.58,65.42,32.29,2.66
2,254,Najrān,40,SA,1592509594,14,17.49,44.13,31.00,1.31
3,265,Lagunas,3,PE,1592509595,42,-5.23,-75.68,34.01,0.51
4,318,Rome,1,US,1592509445,35,43.21,-75.46,31.00,2.60
5,326,Pittsburg,1,US,1592509456,16,38.03,-121.88,33.33,2.60
6,337,Yucca Valley,0,US,1592509315,20,34.11,-116.43,31.67,1.34
7,391,San Ramon,1,US,1592509471,12,37.78,-121.98,33.33,2.60
8,427,Sikeston,1,US,1592509620,40,36.88,-89.59,30.00,1.43
9,490,Ukiah,1,US,1592509630,19,39.15,-123.21,32.78,2.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_data
hotel_df["Hotel Name"]= ""



base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 8000,
    "type": "hotel",
    "key": g_key,
}
for index, row in hotel_df.iterrows():
    latitude = str(row['Lat'])
    longitude = str(row['Lng'])
    target_coor = latitude + ", " + longitude
    params['location'] = target_coor
    #print(f"Retrieving Results for Index {index}: {location}.")
    response = requests.get(base_url, params=params)
    hotels_data = response.json()
    #print(json.dumps(hotels_data, indent=4, sort_keys=True))
    results = hotels_data['results']
    try:
        print(hotels_data["results"][0]["name"])
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Los Pochotes
Kharan
Najran
Pueb Lagunas
Rome
Antioch
Yucca Valley
San Ramon
Sikeston
Ukiah


In [7]:
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,89,Sinaloa,0,MX,1592509359,34,25.00,-107.50,36.67,2.24,Los Pochotes
1,234,Kharan,0,PK,1592509591,10,28.58,65.42,32.29,2.66,Kharan
2,254,Najrān,40,SA,1592509594,14,17.49,44.13,31.00,1.31,Najran
3,265,Lagunas,3,PE,1592509595,42,-5.23,-75.68,34.01,0.51,Pueb Lagunas
4,318,Rome,1,US,1592509445,35,43.21,-75.46,31.00,2.60,Rome
5,326,Pittsburg,1,US,1592509456,16,38.03,-121.88,33.33,2.60,Antioch
6,337,Yucca Valley,0,US,1592509315,20,34.11,-116.43,31.67,1.34,Yucca Valley
7,391,San Ramon,1,US,1592509471,12,37.78,-121.98,33.33,2.60,San Ramon
8,427,Sikeston,1,US,1592509620,40,36.88,-89.59,30.00,1.43,Sikeston
9,490,Ukiah,1,US,1592509630,19,39.15,-123.21,32.78,2.10,Ukiah


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))